# Historical climate for all glaciers (in OGGM format)

In [15]:
# basic
import os

# spatial
import geopandas as gpd
import xarray as xr
import xarray_regrid

period = slice("1980-01-01", "2020-12-31")
path_pmet = "/home/rooda/OneDrive/Projects/PatagoniaMet/"
path_disk = "/home/rooda/Pipeline/DeepHydro/"

In [16]:
# template and source files file
dem        = xr.open_dataset(path_pmet + "GIS/Predictors/dem_patagonia005.tif").rename({'y':'latitude', 'x':'longitude', 'band_data':'hgt'})
dem        = dem.sel(band=1, drop=True).drop_vars("spatial_ref").fillna(0)

# PMET v1.1
pp_pmet    = xr.open_dataset(path_pmet + "data/Zenodo/v11/PP_PMETsim_1980_2020_v11d.nc").sel(time = period)
t2m_pmet   = xr.open_dataset(path_pmet + "data/Zenodo/v11/Tavg_PMETsim_1980_2020_v11d.nc").sel(time = period)

In [ ]:
# actual prepro
dem        = dem.regrid.linear(pp_pmet)
t2m_pmet   = t2m_pmet.regrid.linear(pp_pmet)

pmet   = xr.merge([pp_pmet,  t2m_pmet, dem]).rename({'pp':'prcp', 't2m':'temp'})

pmet.prcp.attrs['units']  = "mm"
pmet.temp.attrs['units']  = "degc"
pmet.hgt.attrs['units']   = "m"

pmet.to_netcdf(path_disk + "CLIMATE/PMETsim_historical_OGGM.nc")